# Make an MTH5 from ZEN data

This notebook provides an example of how to read in ZEN (.Z3D) files into an MTH5.  

In [1]:
from mth5.mth5 import MTH5
from mth5.io.zen import Z3DCollection
from mth5 import read_file

2023-03-27 14:44:02,595 [line 135] mth5.setup_logger - INFO: Logging file can be found C:\Users\jpeacock\OneDrive - DOI\Documents\GitHub\mth5\logs\mth5_debug.log


### Z3D Collection

We will use the `Z3DCollection` to assemble the *.z3d* files into a logical order by schedule action or run. 

**Note**: `n_samples` is an estimate based on file size not the data.  To get an accurate number you should read in the full file.  Same with `start` and `end`. `start` is based on the schedule start time which is usually 2 seconds earlier than the data start because of instrument buffer while chaning sampling rates. `end` is based on file size and sample rate.

The `Z3DCollection.get_runs()` will return a two level ordered dictionary (`OrderedDict`).  The first level is keyed by station ID.  These objects are in turn ordered dictionaries by run ID.  Therefore you can loop over stations and runs.   

In [2]:
zc = Z3DCollection(r"c:\Users\jpeacock\OneDrive - DOI\mt\cl2022\cl471")
runs = zc.get_runs(sample_rates=[4096, 256])
print(f"Found {len(runs)} station with {len(runs[list(runs.keys())[0]])} runs")

Found 1 station with 9 runs


In [3]:
runs["471"]["sr4096_0002"]

,survey,station,run,start,end,channel_id,component,fn,sample_rate,file_size,n_samples,sequence_number,instrument_id,calibration_fn
25,clearlake,471,sr4096_0002,2022-11-02 00:59:57+00:00,2022-11-02 01:09:53.351318+00:00,2,hy,c:\Users\jpeacock\OneDrive - DOI\mt\cl2022\cl4...,4096.0,9644156,2442655,2,ZEN_133,None
26,clearlake,471,sr4096_0002,2022-11-02 00:59:57+00:00,2022-11-02 01:09:53.349365+00:00,5,ey,c:\Users\jpeacock\OneDrive - DOI\mt\cl2022\cl4...,4096.0,9641564,2442647,2,ZEN_133,None
27,clearlake,471,sr4096_0002,2022-11-02 00:59:57+00:00,2022-11-02 01:09:53.352051+00:00,4,ex,c:\Users\jpeacock\OneDrive - DOI\mt\cl2022\cl4...,4096.0,9641608,2442658,2,ZEN_133,None
28,clearlake,471,sr4096_0002,2022-11-02 00:59:57+00:00,2022-11-02 01:09:53.352051+00:00,3,hz,c:\Users\jpeacock\OneDrive - DOI\mt\cl2022\cl4...,4096.0,9644168,2442658,2,ZEN_133,None
29,clearlake,471,sr4096_0002,2022-11-02 00:59:57+00:00,2022-11-02 01:09:53.349854+00:00,1,hx,c:\Users\jpeacock\OneDrive - DOI\mt\cl2022\cl4...,4096.0,9644644,2442649,2,ZEN_133,None


## Build MTH5

Now that we have a logical collection of files, lets load them into an MTH5.  We will simply loop of the stations, runs, and channels in the ordered dictionary.

There are a few things that we need to keep track of.  

- The station metadata pulled directly from the Z3D files can be input into the station metadata, be sure to use the `write_metadata` method to write the metadata to the MTH5.
- The Z3D files have the coil response and zen response embedded in the file, so we can put those into the appropriate filter container in MTH5.  This is important for calibrating later.  
- Since this is a MTH5 file version 0.2.0 the filters are in the `survey_group` so add them there.
- If you want to calibrate the data set calibrate to `True`.  

In [4]:
calibrate = False
new_sample_rate = 1
m = MTH5()
if calibrate:
    m.data_level = 2
m.open_mth5(zc.file_path.joinpath("from_z3d.h5"))

2023-03-27 14:44:08,924 [line 674] mth5.mth5.MTH5._initialize_file - INFO: Initialized MTH5 0.2.0 file c:\Users\jpeacock\OneDrive - DOI\mt\cl2022\cl471\from_z3d.h5 in mode a


In [5]:
survey_group = m.add_survey("CL2022")

In [6]:
%%time

for station_id in runs.keys():
    station_group = survey_group.stations_group.add_station(station_id)
    station_group.metadata.update(zc.station_metadata_dict[station_id])
    station_group.write_metadata()
    decimated_run_list = []
    for run_id, run_df in runs[station_id].items():
        run_group = station_group.add_run(run_id)
        for row in run_df.itertuples():
            ch_ts = read_file(row.fn)
            # NOTE: this is where the calibration occurs
            if calibrate:
                ch_ts = ch_ts.remove_instrument_response()
            run_group.from_channel_ts(ch_ts)
        if new_sample_rate is not None:
            decimated_run_list.append(run_group.to_runts())
    if len(decimated_run_list) > 1:
        long_run = decimated_run_list[0].merge(decimated_run_list[1:], new_sample_rate=new_sample_rate)
        long_run_group = station_group.add_run(f"sr{int(new_sample_rate)}_0001")
        long_run_group.from_runts(long_run)
        

2023-03-27 14:45:33,622 [line 562] mth5.timeseries.run_ts.RunTS.validate_metadata - WARNING: end time of dataset 2022-11-02T01:09:41+00:00 does not match metadata end 2022-11-02T01:09:41.998779+00:00 updating metatdata value to 2022-11-02T01:09:41+00:00
2023-03-27 14:45:34,791 [line 562] mth5.timeseries.run_ts.RunTS.validate_metadata - WARNING: end time of dataset 2022-11-02T06:59:41+00:00 does not match metadata end 2022-11-02T06:59:41.968750+00:00 updating metatdata value to 2022-11-02T06:59:41+00:00
2023-03-27 14:45:36,779 [line 562] mth5.timeseries.run_ts.RunTS.validate_metadata - WARNING: end time of dataset 2022-11-02T07:09:41+00:00 does not match metadata end 2022-11-02T07:09:41.999268+00:00 updating metatdata value to 2022-11-02T07:09:41+00:00
2023-03-27 14:45:37,904 [line 562] mth5.timeseries.run_ts.RunTS.validate_metadata - WARNING: end time of dataset 2022-11-02T12:59:41+00:00 does not match metadata end 2022-11-02T12:59:41.968750+00:00 updating metatdata value to 2022-11-02

Wall time: 1min 57s


In [8]:
%%time
station_group.update_station_metadata()
station_group.write_metadata()

survey_group.update_survey_metadata()
survey_group.write_metadata()

Wall time: 5.13 s


#### MTH5 Structure

Have a look at the MTH5 structure and make sure it looks correct.

In [9]:
m

/:
    |- Group: Experiment
    --------------------
        |- Group: Reports
        -----------------
        |- Group: Standards
        -------------------
            --> Dataset: summary
            ......................
        |- Group: Surveys
        -----------------
            |- Group: CL2022
            ----------------
                |- Group: Filters
                -----------------
                    |- Group: coefficient
                    ---------------------
                        |- Group: dipole_47.00m
                        -----------------------
                        |- Group: dipole_58.00m
                        -----------------------
                        |- Group: zen_counts2mv
                        -----------------------
                    |- Group: fap
                    -------------
                        |- Group: ant4_2314_response
                        ----------------------------
                            --> Dataset: fap_ta

### Channel Summary

Have a look at the channel summary and make sure everything looks good.

In [10]:
m.channel_summary.summarize()
m.channel_summary.to_dataframe()

,survey,station,run,latitude,longitude,elevation,component,start,end,n_samples,sample_rate,measurement_type,azimuth,tilt,units,hdf5_reference,run_hdf5_reference,station_hdf5_reference
0,CL2022,471,sr256_0001,38.986123,-123.068907,245.9,ex,2022-11-01 23:04:58+00:00,2022-11-02 22:44:41+00:00,85184,1.0,electric,0.0,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
1,CL2022,471,sr256_0001,38.986123,-123.068907,245.9,ey,2022-11-01 23:04:58+00:00,2022-11-02 22:44:41+00:00,85184,1.0,electric,0.0,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
2,CL2022,471,sr256_0001,38.986123,-123.068907,245.9,hx,2022-11-01 23:04:58+00:00,2022-11-02 22:44:41+00:00,85184,1.0,magnetic,0.0,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
3,CL2022,471,sr256_0001,38.986123,-123.068907,245.9,hy,2022-11-01 23:04:58+00:00,2022-11-02 22:44:41+00:00,85184,1.0,magnetic,90.0,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
4,CL2022,471,sr256_0001,38.986123,-123.068907,245.9,hz,2022-11-01 23:04:58+00:00,2022-11-02 22:44:41+00:00,85184,1.0,magnetic,0.0,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
5,CL2022,471,sr256_0001,38.986123,-123.068907,245.9,ex,2022-11-01 23:04:58+00:00,2022-11-02 00:59:41.968750+00:00,1762297,256.0,electric,0.0,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
6,CL2022,471,sr256_0001,38.986123,-123.068907,245.9,ey,2022-11-01 23:04:58+00:00,2022-11-02 00:59:41.968750+00:00,1762297,256.0,electric,0.0,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
7,CL2022,471,sr256_0001,38.986123,-123.068907,245.9,hx,2022-11-01 23:04:58+00:00,2022-11-02 00:59:41.968750+00:00,1762297,256.0,magnetic,0.0,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
8,CL2022,471,sr256_0001,38.986123,-123.068907,245.9,hy,2022-11-01 23:04:58+00:00,2022-11-02 00:59:41.968750+00:00,1762297,256.0,magnetic,90.0,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
9,CL2022,471,sr256_0001,38.986123,-123.068907,245.9,hz,2022-11-01 23:04:58+00:00,2022-11-02 00:59:41.968750+00:00,1762297,256.0,magnetic,0.0,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>


## Close the MTH5

This is important, you should close the file after you are done using it.  Otherwise bad things can happen if you try to open it with another program or Python interpreter.

In [11]:
m.close_mth5()

2023-03-27 14:51:53,079 [line 755] mth5.mth5.MTH5.close_mth5 - INFO: Flushing and closing c:\Users\jpeacock\OneDrive - DOI\mt\cl2022\cl471\from_z3d.h5
